## Create directories with date and versions

In [1]:
## Create directories first
t=format(Sys.time(), "%d_%b_%Y")
dir.create(paste("C:/MinuteClinic/Dynamic_time_warping/modeling_work_",t,sep=""))
setwd(paste("C:/MinuteClinic/Dynamic_time_warping/modeling_work_",t,sep=""))
subDir=paste(getwd())
version_path=paste(subDir,"/version_name",sep="")
ifelse(file.exists(version_path),,dir.create(file.path(version_path)))
indata_path=paste(subDir,"/dtw_indata",sep="")
ifelse(file.exists(indata_path),,dir.create(file.path(indata_path)))

modelpath=version_path
outpath=paste(subDir,"/version_name/output",sep="")
ifelse(file.exists(outpath),,dir.create(file.path(outpath)))
#create figures folder
#figures=paste(subDir,"/version_name/figures",sep="")
#ifelse(file.exists(figures),,dir.create(file.path(figures)))

list.files()
setwd(modelpath)
list.files()

[1] TRUE

[1] TRUE

[1] TRUE

[1] "dtw_indata"   "version_name"

[1] "output"

## Create a time series data set from hourly data
Starting with visit level data. Create an hourly aggregated data set for analysis.

In [2]:
require(dplyr)
require(lubridate)
# Develop best version hourly data ------
options(warn=0)
visit_level=read.csv('C:/MinuteClinic/Data/MC_data_01JAN2016_to_02JUL2018.csv')
visit_level$Employee=ifelse(visit_level$VisitOrigin=='Employee',1,0)
visit_level$Hmpil=ifelse(visit_level$VisitOrigin=='Hmpil',1,0)
visit_level$Scheduled=ifelse(visit_level$VisitOrigin=='Scheduled',ifelse(visit_level$ScheduleVisitType=="PSS",1,0),0)
visit_level$WalkIn=ifelse(visit_level$VisitOrigin=='Walk-In',1,0)
visit_level$RevenueGenerating=ifelse(visit_level$VisitStatus=='Revenue Generating',1,0)
visit_level$CompleteNonGenerating=ifelse(visit_level$VisitStatus=='Complete Non Revenue',1,0)
visit_level$ReferOutNoCharge=ifelse(visit_level$VisitStatus=='Refer Out No Charge',1,0)
visit_level$Canceled=ifelse(visit_level$VisitStatus=='Canceled',1,0)
str(visit_level)
visit_level$year=substr(visit_level$AppointmentDate,1,4)
visit_level$day=substr(visit_level$AppointmentDate,1,4)
visit_level$Wait.Time=ifelse(visit_level$Canceled==1,0.01,visit_level$Wait.Time)
visit_level=filter(visit_level, year==2018)
head(visit_level)

Loading required package: dplyr

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Loading required package: lubridate

Attaching package: 'lubridate'

The following object is masked from 'package:base':

    date



'data.frame':	10958515 obs. of  19 variables:
 $ AppointmentDate         : int  20170101 20170101 20170101 20170101 20170101 20170101 20170101 20170101 20170101 20170101 ...
 $ ClinicNumber            : Factor w/ 1190 levels "00012","00015",..: 552 552 552 908 1073 1073 1075 1076 1077 1078 ...
 $ ProviderEpicid          : Factor w/ 3996 levels "*Unspecified",..: 272 1 272 201 2920 2920 2285 2789 2283 2884 ...
 $ VisitStatus             : Factor w/ 7 levels "Canceled","Complete Non Revenue",..: 6 4 6 6 6 6 6 6 6 6 ...
 $ AppointmentTimeOfDay    : int  123000 141000 151000 133000 115000 123000 91000 142500 104000 100000 ...
 $ AppointmentMadeTimeOfDay: int  122724 141127 151106 132939 114259 122823 91139 142612 104557 95952 ...
 $ Wait.Time               : num  0.05 NA 18.983 0.05 0.217 ...
 $ ScheduleVisitType       : Factor w/ 5 levels "","ClevelandClinicCoMan",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ VisitOrigin             : Factor w/ 4 levels "Employee","Hmpil",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ 

AppointmentDate,ClinicNumber,ProviderEpicid,VisitStatus,AppointmentTimeOfDay,AppointmentMadeTimeOfDay,Wait.Time,ScheduleVisitType,VisitOrigin,ScheduleTimeOfDay,...,Employee,Hmpil,Scheduled,WalkIn,RevenueGenerating,CompleteNonGenerating,ReferOutNoCharge,Canceled,year,day
20180101,03631,*Unspecified,Canceled,161000,161000,0.010000,PSS,Scheduled,1610,...,0,0,1,0,0,0,0,1,2018,2018
20180101,03648,*Unspecified,Canceled,110000,110000,0.010000,TOS,Scheduled,1100,...,0,0,0,0,0,0,0,1,2018,2018
20180101,06633,642726,Revenue Generating,94000,93651,0.066666,,Walk-In,940,...,0,0,0,1,1,0,0,0,2018,2018
20180101,06633,642726,Revenue Generating,113000,112448,0.116666,,Walk-In,1130,...,0,0,0,1,1,0,0,0,2018,2018
20180101,06633,642726,Revenue Generating,115000,114920,0.216666,,Walk-In,1150,...,0,0,0,1,1,0,0,0,2018,2018
20180101,06633,642726,Revenue Generating,124000,123823,0.083333,,Walk-In,1240,...,0,0,0,1,1,0,0,0,2018,2018


In [8]:
#convert visit level to hourly
visit_level$Hour=visit_level$AppointmentTimeOfDay/10000
hourly=data.frame(visit_level%>%group_by(ClinicNumber,AppointmentDate,Hour)%>%summarise(provider.count=n_distinct(ProviderEpicid),
        Employee_Sum=sum(Employee),Hmpil_Sum=sum(Hmpil),Scheduled_Sum=sum(Scheduled),WalkIn_sum=sum(WalkIn),
        ReferOutNoCharge_Sum=sum(ReferOutNoCharge),Canceled_Sum=sum(Canceled),wait_time_avg=mean(Wait.Time,na.rm=T)))
head(hourly)

ClinicNumber,AppointmentDate,Hour,provider.count,Employee_Sum,Hmpil_Sum,Scheduled_Sum,WalkIn_sum,ReferOutNoCharge_Sum,Canceled_Sum,wait_time_avg
00012,20180101,8.50,1,0,0,0,1,0,0,2.10000
00012,20180101,9.15,1,0,1,0,0,0,1,0.01000
00012,20180101,9.40,1,0,0,0,1,0,0,2.75000
00012,20180101,10.00,1,0,0,0,1,0,0,24.78333
00012,20180101,10.15,1,0,0,0,1,0,0,9.00000
00012,20180101,10.30,1,0,0,0,1,0,0,20.88333


In [18]:
hourly=filter(hourly, Hour>8 & Hour<=20)
hourly$Hour=round(hourly$Hour)
range(hourly$Hour)
#hist(hourly$Hour)

[1]  8 20

In [23]:
hourly$AllVisits=hourly$Employee_Sum+hourly$Hmpil_Sum+hourly$Scheduled_Sum+hourly$WalkIn_sum
daily=hourly%>%group_by(ClinicNumber,AppointmentDate)%>%summarise(daily_sum=sum(AllVisits))
hrly=left_join(select(hourly,ClinicNumber, AppointmentDate,Hour,AllVisits), daily, by=c("ClinicNumber","AppointmentDate"))
hrly=unique(hrly)
hrly$PerVisits=hrly$AllVisits/hrly$daily_sum
head(hrly)

,ClinicNumber,AppointmentDate,Hour,AllVisits,daily_sum,PerVisits
1,00012,20180101,8,1,17,0.05882353
2,00012,20180101,9,1,17,0.05882353
4,00012,20180101,10,1,17,0.05882353
7,00012,20180101,11,1,17,0.05882353
10,00012,20180101,12,1,17,0.05882353
15,00012,20180101,13,1,17,0.05882353


In [30]:
summary(hrly)
#plot(hrly$Hour,hrly$PerVisits)
#hist(hrly$PerVisits)
getwd()
write.csv(hrly, 'C:/MinuteClinic/Dynamic_time_warping/DTW_Input_data/hourly_percent_visits_2018.csv', row.names=F)

  ClinicNumber     AppointmentDate         Hour         AllVisits     
 09721  :   2972   Min.   :20180101   Min.   : 8.00   Min.   : 0.000  
 05924  :   2736   1st Qu.:20180209   1st Qu.:10.00   1st Qu.: 1.000  
 09795  :   2719   Median :20180325   Median :13.00   Median : 1.000  
 02175  :   2709   Mean   :20180353   Mean   :13.21   Mean   : 1.042  
 09963  :   2634   3rd Qu.:20180512   3rd Qu.:16.00   3rd Qu.: 1.000  
 01545  :   2590   Max.   :20180702   Max.   :20.00   Max.   :44.000  
 (Other):1847779                                                      
   daily_sum        PerVisits     
 Min.   :  0.00   Min.   :0.0000  
 1st Qu.: 14.00   1st Qu.:0.0385  
 Median : 20.00   Median :0.0526  
 Mean   : 20.89   Mean   :0.0604  
 3rd Qu.: 26.00   3rd Qu.:0.0714  
 Max.   :161.00   Max.   :1.0000  
                  NA's   :388     

[1] "C:/MinuteClinic/Dynamic_time_warping/modeling_work_31_Jul_2018/version_name"

In [46]:
memory.limit()
rm(list=ls())
gc()

[1] 7603

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2543132,135.9,4391388,234.6,6861544,366.5
Vcells,808381246,6167.5,1193888843,9108.7,918528243,7007.9


In [47]:
#select just clinic and percent
#convert data to one clinic per column of just the percent visits
hrly=read.csv('C:/MinuteClinic/Dynamic_time_warping/DTW_Input_data/hourly_percent_visits_2018.csv')
hrly=select(hrly,ClinicNumber,PerVisits)
numcurves=length(unique(hrly$ClinicNumber))
totallength=length(hrly$ClinicNumber)
library(tidyr)
clinic_list=paste(as.character(unique(hrly$ClinicNumber)),sep=",")
wt=hrly %>% separate(PerVisits, clinic_list)
head(wt)
#widetable <- data.frame(matrix(hrly$PerVisits,ncol = numcurves, byrow = F))
#head(widetable)

#dmatrix=(dist(widetable, method="DTW"))
#clusters=(hclust(dmatrix, method="average"))
#ct=cutree(clusters,10)
#zt=do.call(cbind,list(rep(ct, each=(totallenght/numcurves))))

ERROR: Error: cannot allocate vector of size 14.2 Mb
